# import


In [361]:
import numpy as np
import pandas as pd
import io
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Input, LSTM, concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn. preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# 1. 데이터

In [362]:
# PATH = '/content/drive/MyDrive/'
PATH = 'C:/study/keras/keras_data/stock/'

samsung = pd.read_csv(PATH + '삼성전자 주가.csv', header=0, index_col=None, sep=',', encoding='cp949', thousands=',').loc[::-1]
# print(samsung)
# print(samsung.shape) #(1980, 17)

amore = pd.read_csv(PATH + '아모레퍼시픽 주가.csv', header=0, index_col=None, sep=',', encoding='cp949', thousands=',').loc[::-1]
# print(amore)
# print(amore.shape)   #(2220, 17)

# 삼성전자 x ,y 추출
samsung_x = samsung[['고가', '저가','종가', '외인(수량)', '기관']]
samsung_y = samsung[['시가']].to_numpy() # x 데이터는 아래에서 split 함수를 거치며 numpy array로 변환되므로 y는 여기서 변환해준다
# print(samsung_x)
# print(samsung_y)
# print(samsung_x.shape) # (1980, 5)
# print(samsung_y.shape) # (1980, 1)

# 아모레 x, y 추출
amore_x = amore.loc[1979:0,['고가', '저가', '종가', '외인(수량)', '시가']]
# print(amore_x)
# print(amore_x.shape) #(1980, 5)

samsung_x = MinMaxScaler().fit_transform(samsung_x)
amore_x = MinMaxScaler().fit_transform(amore_x)

def split_data(dataset, timesteps):
    tmp = []
    for i in range(len(dataset) - timesteps + 1):
        subset = dataset[i : (i + timesteps)]
        tmp.append(subset)
    return np.array(tmp)

samsung_x = split_data(samsung_x, 5)
amore_x = split_data(amore_x, 5)
# print(samsung_x.shape) #(1976, 5, 5)
# print(amore_x.shape) #(1976, 5, 5)

samsung_y = samsung_y[4:, :] # x 데이터와 shape을 맞춰주기 위해 4개 행 제거
# print(samsung_y.shape) #(1976, 1)

# 예측에 사용할 데이터 추출 (마지막 값)
samsung_x_predict = samsung_x[-1].reshape(-1, 5, 5)
amore_x_predict = amore_x[-1].reshape(-1, 5, 5)
# print(samsung_x_predict.shape) # (5, 5, 1)
# print(amore_x_predict.shape) # (5, 5, 1)

samsung_x_train, samsung_x_test, samsung_y_train, samsung_y_test, amore_x_train, amore_x_test  = train_test_split(
    samsung_x, samsung_y, amore_x, train_size=0.7, random_state=333)

print(samsung_x_train.shape, samsung_x_test.shape)  # (1383, 5, 5) (593, 5, 5)
print(samsung_y_train.shape, samsung_y_test.shape) # (1383, 1) (593, 1)
print(amore_x_train.shape, amore_x_test.shape)  # (1383, 5, 5) (593, 5, 5)

(1383, 5, 5) (593, 5, 5)
(1383, 1) (593, 1)
(1383, 5, 5) (593, 5, 5)


# 2. 모델

In [363]:
# 삼성전자
samsung_input = Input(shape=(5, 5))
samsung_layer1 = LSTM(64, return_sequences=True, activation='relu')(samsung_input)
samsung_layer1 = Dropout(0.2)(samsung_layer1)
samsung_layer2 = LSTM(128, activation='relu')(samsung_layer1)
samsung_layer3 = Dense(256, activation='relu')(samsung_layer2)
samsung_layer4 = Dropout(0.2)(samsung_layer3)
samsung_layer5 = Dense(1024, activation='relu')(samsung_layer4)
samsung_layer6 = Dense(128, activation='relu')(samsung_layer5)
samsung_layer6 = Dropout(0.2)(samsung_layer6)
samsung_layer7 = Dense(32, activation='relu')(samsung_layer6)
samsung_output = Dense(1)(samsung_layer7)

# 아모레퍼시픽
amore_input = Input(shape=(5, 5))
amore_layer1 = LSTM(64, return_sequences=True, activation='relu')(amore_input)
amore_layer1 = Dropout(0.2)(amore_layer1)
amore_layer2 = LSTM(128, activation='relu')(amore_layer1)
amore_layer3 = Dense(256, activation='relu')(amore_layer2)
amore_layer4 = Dropout(0.2)(amore_layer3)
amore_layer5 = Dense(1024, activation='relu')(amore_layer4)
amore_layer6 = Dense(128, activation='relu')(amore_layer5)
amore_layer6 = Dropout(0.2)(samsung_layer6)
amore_layer7 = Dense(32, activation='relu')(amore_layer6)
amore_output = Dense(1)(amore_layer7)

# 병합
merge_layer1 = concatenate([samsung_output, amore_output])
merge_layer2 = Dense(64, activation='relu')(merge_layer1)
merge_layer3 = Dense(128, activation='relu')(merge_layer2)
merge_layer4 = Dense(256, activation='relu')(merge_layer3)
merge_layer5 = Dense(512, activation='relu')(merge_layer4)
merge_layer6 = Dense(32, activation='relu')(merge_layer5)
merge_output = Dense(1, activation='relu')(merge_layer6)

model = Model(inputs=[samsung_input, amore_input], outputs=[merge_output])
model.summary()

Model: "model_69"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_143 (InputLayer)         [(None, 5, 5)]       0           []                               
                                                                                                  
 lstm_284 (LSTM)                (None, 5, 64)        17920       ['input_143[0][0]']              
                                                                                                  
 dropout_426 (Dropout)          (None, 5, 64)        0           ['lstm_284[0][0]']               
                                                                                                  
 lstm_285 (LSTM)                (None, 128)          98816       ['dropout_426[0][0]']            
                                                                                           

# 3. 컴파일, 훈련



In [364]:
model.compile(loss='mse', optimizer= 'adam')

ES = EarlyStopping(monitor='val_loss', mode='auto', patience=32, restore_best_weights=True)
MCP = ModelCheckpoint(monitor='val_loss', mode='auto', save_best_only=True,filepath = PATH + 'stock_ModelCheckPoint.hdf5') # 모델과 가중치 저장

model.fit([samsung_x_train, amore_x_train], samsung_y_train , epochs=1024, batch_size=1024, validation_split=0.2, callbacks=[ES, MCP])

model.save_weights(PATH + 'stock_weight.h5') # 가중치만 저장

Epoch 1/1024
2/2 [==============================] - 2s 517ms/step - loss: 1419245125632.0000 - val_loss: 1492798406656.0000
Epoch 2/1024
2/2 [==============================] - 0s 210ms/step - loss: 1419245125632.0000 - val_loss: 1492798144512.0000
Epoch 3/1024
2/2 [==============================] - 0s 213ms/step - loss: 1419244732416.0000 - val_loss: 1492797095936.0000
Epoch 4/1024
2/2 [==============================] - 0s 202ms/step - loss: 1419243683840.0000 - val_loss: 1492794474496.0000
Epoch 5/1024
2/2 [==============================] - 0s 193ms/step - loss: 1419240669184.0000 - val_loss: 1492787134464.0000
Epoch 6/1024
2/2 [==============================] - 0s 191ms/step - loss: 1419233591296.0000 - val_loss: 1492767211520.0000
Epoch 7/1024
2/2 [==============================] - 0s 191ms/step - loss: 1419212881920.0000 - val_loss: 1492714389504.0000
Epoch 8/1024
2/2 [==============================] - 0s 188ms/step - loss: 1419160190976.0000 - val_loss: 1492572962816.0000
Epoch 9/

# 4. 평가, 예측

In [365]:
loss=model.evaluate([samsung_x_test, amore_x_test], samsung_y_test, batch_size=1024)
samsung_y_predict = model.predict([samsung_x_predict, amore_x_predict])

print("loss : ", loss)
print("삼성전자 시가 :" , samsung_y_predict)

1/1 [==============================] - 0s 26ms/step - loss: 767067840.0000
loss :  767067840.0
삼성전자 시가 : [[62264.87]]
